In [105]:
import glob
from os import path
import json
import numpy as np
from scipy.stats import sem
import pandas as pd

In [106]:
abl_fp = "/data/b2p-siteident/experiments/new_baseline_mods"

In [112]:
header = [
    "data_mod", "version", "freeze", "use_several_samples", "tile_size"
]
metrics = [
    # "val_acc", "val_weighted_f1",
    # "test_rw_balanced__acc",
    # "test_rw_weighted_f1",
    # "test_ug_balanced_acc", 
    # "test_ug_weighted_f1",
    "ecount"
]

entries_dict = {}
for exp_fp in sorted(glob.glob(path.join(abl_fp, "*"))):
    if not (path.isfile(path.join(exp_fp, "stats_True_32.json")) and
            path.isfile(path.join(exp_fp, "stats_False_32.json"))):
            print("{} not finished.".format(exp_fp))
            continue
    with open(path.join(exp_fp, "opts.json")) as f:
        opts = json.load(f)
    
    with open(path.join(exp_fp, "stats_True_32.json")) as f:
        stats_no_several = json.load(f)
    
    with open(path.join(exp_fp, "stats_False_32.json")) as f:
        stats_several = json.load(f)
    
    data_mod = "small" if len(opts["data_modalities"]) < 7 else "large"
    data_version = opts["data_version"]
    freeze = opts["use_last_n_layers"]
    tile_size = opts["tile_size"]
    for use_several in [True, False]:
        if use_several:
            stats = stats_several
        else:
            stats = stats_no_several
        key = (data_mod, data_version, freeze, use_several, tile_size)
        if key not in entries_dict:
            entries_dict[key] = {m: [] for m in metrics}
            entries_dict[key]["ecount"] = 0
            entries_dict[key]["path"] = exp_fp
        for m in metrics:
            if m == "ecount":
                continue
            entries_dict[key][m].append(stats[m])
        entries_dict[key]["ecount"] += 1

entries = []
i = 0
for k, entry in entries_dict.items():
    avg_entry = list(k)
    for m in metrics:
        if m == "ecount":
            avg_entry.append(entry[m])
        else:
            val = np.array(entry[m])
            if "acc" in m:
                val *= 100
            avg_entry.append(round(np.mean(val), 2))
            avg_entry.append(round(sem(val), 2))
    entries.append(avg_entry)
    i += 1

metrics_header = []
for m in metrics:
    if m == "ecount":
        metrics_header.append(m)
    else:
        metrics_header.append(m + "_m")
        metrics_header.append(m + "_ste")
df = pd.DataFrame(entries, columns=header + metrics_header)

KeyError: 'test_balanced_acc'

In [110]:
df[(df.tile_size == 1200) & (df.use_several_samples == True) & (df.freeze == 9)].sort_values(
    by=["data_mod", "tile_size", "version", "freeze"])

,data_mod,version,freeze,use_several_samples,tile_size,test_rw_acc_m,test_rw_acc_ste,test_ug_acc_m,test_ug_acc_ste,ecount
2,large,v1,9,True,1200,61.80,2.67,68.60,0.76,3
10,large,v2,9,True,1200,58.65,2.64,81.51,1.71,3
6,small,v1,9,True,1200,65.00,1.33,66.07,0.47,3
14,small,v2,9,True,1200,62.52,0.43,78.91,1.56,3


In [ ]:
\res{65.00}{1.33} & \res{66.07}{0.47} & & & \res{62.52}{0.43} &\res{78.91}{1.56} &